# Part 1: Data Preprocessing

Dataset link: [Predicting Churn for Bank Customers](https://www.kaggle.com/datasets/adammaus/predicting-churn-for-bank-customers?select=Churn_Modelling.csv)

## Importing the libraries

In [127]:
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten, Convolution2D
from keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam

## Dealing dataset

In [128]:
dataset = pd.read_csv('_train.csv')

In [129]:
dataset

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15633608,Black,641,France,Male,33,2,146193.60,2,1,1,55796.83,1
1,2,15742323,Barese,541,France,Male,39,7,0.00,2,1,0,19823.02,0
2,3,15760244,Ives,590,France,Female,76,5,160979.68,1,0,1,13848.58,0
3,4,15600974,He,516,Spain,Male,50,5,0.00,1,0,1,146145.93,1
4,5,15663234,Bishop,508,France,Female,60,7,143262.04,1,1,1,129562.74,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,16996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
16996,16997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
16997,16998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
16998,16999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [130]:
dataset = dataset.drop(columns=['id', 'CustomerId', 'Surname'])

In [131]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,641,France,Male,33,2,146193.60,2,1,1,55796.83,1
1,541,France,Male,39,7,0.00,2,1,0,19823.02,0
2,590,France,Female,76,5,160979.68,1,0,1,13848.58,0
3,516,Spain,Male,50,5,0.00,1,0,1,146145.93,1
4,508,France,Female,60,7,143262.04,1,1,1,129562.74,0


In [132]:
dataset.shape

(17000, 11)

In [133]:
# detect missing data
dataset.isnull().values.any()

False

In [134]:
dataset.select_dtypes(include='object').columns

Index(['Geography', 'Gender'], dtype='object')

In [135]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [136]:
dataset['Gender'].unique()

array(['Male', 'Female'], dtype=object)

In [137]:
dataset = pd.get_dummies(data=dataset, drop_first=True)

In [138]:
dataset

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,641,33,2,146193.60,2,1,1,55796.83,1,0,0,1
1,541,39,7,0.00,2,1,0,19823.02,0,0,0,1
2,590,76,5,160979.68,1,0,1,13848.58,0,0,0,0
3,516,50,5,0.00,1,0,1,146145.93,1,0,1,1
4,508,60,7,143262.04,1,1,1,129562.74,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16995,771,39,5,0.00,2,1,0,96270.64,0,0,0,1
16996,516,35,10,57369.61,1,1,1,101699.77,0,0,0,1
16997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0
16998,772,42,3,75075.31,2,1,0,92888.52,1,1,0,1


In [139]:
x = dataset.drop(columns='Exited').values

In [140]:
y = dataset['Exited'].values

In [141]:
x.shape, y.shape

((17000, 11), (17000,))

## Splitting the training data and testing data

In [142]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [143]:
x_train = x
y_train = y

In [144]:
x_train.shape, x_test.shape

((17000, 11), (3400, 11))

## Feature Scaling

In [145]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Part 2: Building the model

## TensorFlow implementation

In [146]:
result_acc = []
result_thr = []

In [147]:
## config
def get_model(config):
    layers = [
        Dense(units=u, activation='relu') for u in config
    ]
    layers.append(Dense(units=1, activation='sigmoid'))
    return Sequential(layers)

In [148]:
def accuracy(y_hat, y_test, threshold=0.5):
    y_hat = ((y_hat >= threshold)*1).astype('int64')
    return (y_test == y_hat).sum() / len(y_hat)

In [149]:
def train_eval(model, x_train, x_test, y_train, y_test):
    y_hat = model.predict(x_test).flatten()
    
    acc_max = 0
    threshold_optim = 0

    thresholds = np.arange(0.1,1, step = 0.01)
    acc = np.zeros_like(thresholds)

    for i, threshold in enumerate(thresholds):
        acc[i] = accuracy(y_hat, y_test, threshold)
        if acc_max < acc[i]:
            acc_max = acc[i]
            threshold_optim = thresholds[i]

    result_acc.append(acc_max)
    result_thr.append(threshold_optim)

In [150]:
models = [
    get_model(config) for config in 
    [
        [52, 36, 35, 34, 33, 9, 8],
        [25, 15, 10, 1],
        [55, 51, 45, 33, 20, 2, 1],
        [53, 47, 14, 13, 9, 8, 5, 1],
        [39, 34, 29, 28, 17, 6, 5, 1]
    ]
]

[
    # model.compile(optimizer=Adam(learning_rate=1e-3), loss=SparseCategoricalCrossentropy(from_logits=True)) for model in models
    model.compile(loss=BinaryCrossentropy()) for model in models
]

[
    model.fit(x_train, y_train, epochs=150) for model in models
]

[
    train_eval(model, x_train, x_test, y_train, y_test) for model in models
]

Epoch 1/150
532/532 [==============================] - 3s 3ms/step - loss: 0.3918
Epoch 2/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3456
Epoch 3/150
532/532 [==============================] - 1s 3ms/step - loss: 0.3364
Epoch 4/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3317
Epoch 5/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3258
Epoch 6/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3235
Epoch 7/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3182
Epoch 8/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3141
Epoch 9/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3101
Epoch 10/150
532/532 [==============================] - 1s 3ms/step - loss: 0.3066
Epoch 11/150
532/532 [==============================] - 1s 2ms/step - loss: 0.3037
Epoch 12/150
532/532 [==============================] - 1s 2ms/step - loss: 0.2998
Epoch 13/150


[None, None, None, None, None]

In [151]:
result_acc

[0.9529411764705882,
 0.8976470588235295,
 0.9485294117647058,
 0.9302941176470588,
 0.8017647058823529]

# Part 3: Answer

## Dealing dataset

In [152]:
dataset_ans = pd.read_csv('test.csv')

In [153]:
dataset_ans = dataset_ans.drop(columns=['id', 'CustomerId', 'Surname'])

In [154]:
dataset_ans = pd.get_dummies(data=dataset_ans, drop_first=True)

In [155]:
dataset_ans

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,597,35,8,131101.04,1,1,1,192852.67,1,0,0
1,523,40,2,102967.41,1,1,0,128702.10,0,0,0
2,706,42,8,95386.82,1,1,1,75732.25,0,1,0
3,788,32,4,112079.58,1,0,0,89368.59,0,0,1
4,706,38,5,163034.82,2,1,1,135662.17,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2995,530,41,4,0.00,2,0,1,147606.71,0,1,0
2996,639,77,6,80926.02,2,1,1,55829.25,0,0,1
2997,685,40,2,168001.34,2,1,1,167400.29,0,0,1
2998,692,69,10,154953.94,1,1,1,70849.47,0,0,1


In [156]:
x_ans = dataset_ans.values

In [157]:
x_ans.shape

(3000, 11)

## Feature Scaling

In [158]:
x_ans = sc.transform(x_ans)

## Answer

In [159]:
def standardized_y(y_hat, threshold = 0.9):
    y_hat = ((y_hat >= threshold)*1).astype('int64')
    return y_hat

In [160]:
acc_max = 0
i = 0
for acc in result_acc:
    if acc_max < acc:
        acc_max = acc
        id_optim = i
    i = i + 1

In [161]:
id_optim, result_acc[id_optim]

(0, 0.9529411764705882)

In [162]:
y_ans = models[id_optim].predict(x_ans).flatten()
y_ans = standardized_y(y_ans, result_thr[id_optim])

94/94 [==============================] - 0s 2ms/step


In [163]:
for i in range(len(y_ans)):
    print(f"{y_ans[i]:.0f}")

1
0
0
0
0
1
0
0
1
1
0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
1
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
0
0
0
1
0
0
0
1
1
0
1
0
0
0
1
1
0
0
1
0
0
0
1
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
1
0
1
0
0
0
0
1
0
0
1
0
1
0
1
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
0
0
1
1
1
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
0
0
1
0
1
1
0
0
0
0
0
0
0
0


In [164]:
# get_ans(model_optimal, threshold_optimal, x_ans)